Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Interactive LLM Agent for coding

FLAML offers a suite of interactive LLM agents, which can be used to solve various tasks.

In this notebook, we demonstrate how to use `PythonAgent` and `UserProxyAgent` to write code and execute the code. Here `PythonAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `PythonAgent`. By setting `user_interaction_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `PythonAgent`. For example, when `user_interaction_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `PythonAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `PythonAgent` directly and returns the execution results (success or failure and corresponding outputs) to `PythonAgent`.  

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option:
```bash
pip install flaml[openai]==1.2.4
```

In [1]:
# %pip install flaml[openai]==1.2.4

### Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of  Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure Open API key + base.


In [ ]:
# import openai
# openai.api_key_path = "key.txt"

### Example Task: Create and execute a python script with agents

In the example below, let's see how to use the agents in FLAML to write a python script and execute the script.

In [ ]:
from flaml.autogen.agent.coding_agent import PythonAgent
from flaml.autogen.agent.user_proxy_agent import UserProxyAgent
config_list = oai.config_list_gpt4_gpt35(key_file_path=KEY_LOC)
# the purpose of the following two lines is to log the conversation history
conversations = {}
oai.ChatCompletion.start_logging(conversations)

# create an assistant which is essentially a PythonAgent instance named "coding_agent"
assistant = PythonAgent("coding_agent", request_timeout=600, seed=42, config_list=config_list, model="gpt-3.5-turbo")
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    "user",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply= 10,
    is_termination_msg=lambda x: x.rstrip().endswith("TERMINATE"),
    )
# the assistant receives a message from the user, which contains the task description
assistant.receive(
"""Create and execute a script to plot a rocket without using matplotlib""",
user,
)

Next, let's see how to use the agents to first write the generated script to a file and then execute the script in two sessions of conversation between the `PythonAgent` and the `UserProxyAgent`.

In [ ]:
# it is suggested to reset the assistant to clear the state if the new task is not related to the previous one.
assistant.reset()
assistant.receive(
"""Create a temp.py file with the following content:
```
print('Hello world!')
```""",
user,
)
print(conversations)

oai.ChatCompletion.start_logging(compact=False)
agent.receive("""Execute temp.py""", user)
print(oai.ChatCompletion.logged_history)
oai.ChatCompletion.stop_logging()